In [1]:
# import pandas as pd
import MagmaPandas as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import meltInc.plotting as p
from functools import partial
from MagmaPandas.geochemistry.fO2 import fO2_QFM
from MagmaPandas.geochemistry.Kd_ol_melt import Kd_FeMg_vectorised
from MagmaPandas.geochemistry.Fe_redox import Fe_redox
from MagmaPandas.geochemistry.PEC import Fe_equilibrate, PEC_configuration, PEC_olivine
from MagmaPandas.geochemistry.volatile_solubility import IaconoMarziano as im

# p.layout()

PEC_configuration.print()
# print("\n")
# im.IaconoMarziano_configuration.print()


####### Post-entrapment crystalllisation #######
############### correction model ###############

Settings_______________________________________
Stepsize...................................0.002
Decrease factor............................5    
FeO convergence............................0.05 
Kd convergence.............................0.001
Temperature convergence....................0.1  


In [2]:
melt = mp.read_melt_inclusion('./melt.csv', index_col=['name'], total_col='total')
inclusions = melt.iloc[0:10,:].copy()

In [3]:
forsterite_host = 0.90
P_bar = 1e3

In [ ]:
x, oc, kd = inclusions._Fe_equilibrate(forsterite_host, P_bar, stepsize=0.001)


In [ ]:
o_crystallised = []
for _, inclusion in inclusions.iterrows():
    _, OC, _ = Fe_equilibrate(inclusion, forsterite_host, P_bar, stepsize=0.001)
    o_crystallised.append(OC)
o_crystallised

In [ ]:
oc

In [4]:
t = PEC_olivine(inclusions, forsterite_host, P_bar, FeO_target=10)

In [6]:
t.Fe_loss

name
PI054-07-06    True
PI054-07-05    True
PI056-04-02    True
PI116-04-01    True
PI056-05-01    True
PI089-03-01    True
PI074-05-01    True
PI052-02-01    True
PI056-04-03    True
PI085-04-02    True
dtype: bool

In [ ]:
PEC_configuration.stepsize = 0.001
PEC_configuration.print()
x2, ocx, kd2 = t.Fe_equilibrate()

In [ ]:
ocx